# Origins of path integration in CA1
Compare portions of time/distance cells vs place cells in light/dark environments, or tone vs no tone conditions

### TODO:
- Plot timecourse (LT1, LT2, LT3, toneLT1, toneLT2, toneLT3, seqLT1, seqLT2, etc), average for all animals
- Normalize per animal!
- Information content (AMI distribution) vs specific cell tuning (portion time cells, portion place cells, etc over time)

In [1]:
# Imports
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('plot_style.mplstyle')

import yaml
import numpy as np
np.random.seed(42)
import pingouin as pg
import pandas as pd
import h5py
import os
from functions.signal_processing import smooth_1D
from tqdm import tqdm

# Params
with open('params.yaml','r') as file:
    params = yaml.full_load(file)

In [5]:
sessionList=os.listdir(os.path.join(params['path_to_results'],'tuning_data'))
data_list = [] # cell, animal,day,condition,AMI,pvalue
sessionList=os.listdir(os.path.join(params['path_to_results'],'tuning_data'))
for session in tqdm(sessionList):
    if os.path.exists(os.path.join(params['path_to_results'],'tuning_data',session,'info.yaml')): # TODO: Report any file missing
        info_file=open(os.path.join(params['path_to_results'],'tuning_data',session,'info.yaml'),'r')
        session_info = yaml.full_load(info_file)
        spatial_file = h5py.File(os.path.join(params['path_to_results'],'tuning_data',session,'spatial_tuning.h5'), 'r')
        temporal_file = h5py.File(os.path.join(params['path_to_results'],'tuning_data',session,'temporal_tuning.h5'), 'r')
        distance_file = h5py.File(os.path.join(params['path_to_results'],'tuning_data',session,'distance_tuning.h5'), 'r')
        direction_file = h5py.File(os.path.join(params['path_to_results'],'tuning_data',session,'direction_tuning.h5'), 'r')
        velocity_file = h5py.File(os.path.join(params['path_to_results'],'tuning_data',session,'velocity_tuning.h5'), 'r')

        for i in range(session_info['numNeurons']):
            data_list.append( #This will create one list entry per cell
                {
                    # Basic conditions
                    'subject':session_info['subject'],
                    'region':session_info['region'],
                    'day':session_info['day'],
                    'condition':session_info['condition'],
                    'task':session_info['task'],
                    'darkness':session_info['darkness'],
                    'optoStim':session_info['optoStim'],
                    'rewards':session_info['rewards'],
                    # Info metrics
                    'spatial_info':spatial_file['AMI'][i],
                    'temporal_info':temporal_file['AMI'][i],
                    'distance_info':distance_file['AMI'][i],
                    'velocity_info':velocity_file['AMI'][i],
                    'heading_info':direction_file['AMI'][i],
                    'spatial_pvalue':spatial_file['p_value'][i],
                    'temporal_pvalue':temporal_file['p_value'][i],
                    'distance_pvalue':distance_file['p_value'][i],
                    'velocity_pvalue':velocity_file['p_value'][i],
                    'heading_pvalue':direction_file['p_value'][i]
                }
            )
        
        # Close files
        info_file.close()
        temporal_file.close()
        spatial_file.close()
        distance_file.close()
        direction_file.close()
        velocity_file.close()

df = pd.DataFrame(data_list)

100%|██████████| 256/256 [00:22<00:00, 11.37it/s]


In [54]:
## Create sub-data frames
CA1_df = df.query("region=='CA1' and rewards==True and condition=='normal'")

# Compare tone vs no tone linear track
tone_df = CA1_df.query("task=='legoLT' or task == 'legoSeqLT'")

# Compare dark vs light conditions
dark_df = CA1_df.query("task=='legoLT'")

In [ ]:
# Spatial coding with or without tones
plt.figure(figsize=(.75,.75))
sns.histplot(LT_spatial_AMI[LT_spatial_pvalue<p_val_threshold],
             bins=100,
             stat='probability',
            #  cumulative=True,
             element='poly',
             alpha=.1,
             color='k',
             label='no tones')
sns.histplot(seqLT_spatial_AMI[seqLT_spatial_pvalue<p_val_threshold],
             bins=100,
             stat='probability',
            #  cumulative=True,
             element='poly',
             alpha=.1,
             color='C0',
             label='tones')
plt.plot([np.mean(LT_spatial_AMI[LT_spatial_pvalue<p_val_threshold]),np.mean(LT_spatial_AMI[LT_spatial_pvalue<p_val_threshold])],[0,.1],c='k')
plt.plot([np.mean(seqLT_spatial_AMI[seqLT_spatial_pvalue<p_val_threshold]),np.mean(seqLT_spatial_AMI[seqLT_spatial_pvalue<p_val_threshold])],[0,.1],c='C0')
#plt.xscale('log')
plt.title('Spatial tuning')
plt.xlabel('AMI (log scale)')
plt.xlim([0.0005,.05])
plt.legend(bbox_to_anchor=(1.1, 1), loc='upper left', borderaxespad=0)
plt.savefig(os.path.join(params['path_to_results'],'figures', 'cum_dens_elapsed_spatial_noTone_vs_tone.pdf'))

In [ ]:
# Temporal coding with or without tones
plt.figure(figsize=(.5,.75))
sns.barplot([LT_spatial_AMI[LT_spatial_pvalue<p_val_threshold],seqLT_spatial_AMI[seqLT_spatial_pvalue<p_val_threshold]],
             label=['no tones','tones'],
             errorbar='se',
             capsize=.5,
             palette=['k', 'C0']
             )
plt.title('Spatial tuning')
plt.ylabel('AMI')
plt.xticks([])
plt.legend(bbox_to_anchor=(1.1, 1), loc='upper left', borderaxespad=0)
plt.savefig(os.path.join(params['path_to_results'],'figures', 'bar_elapsed_spatial_noTone_vs_tone.pdf'))

In [ ]:
pg.ttest(x=LT_spatial_AMI[LT_spatial_pvalue<p_val_threshold],
         y=seqLT_spatial_AMI[seqLT_spatial_pvalue<p_val_threshold])